In [1]:
import pandas as pd
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
notebook_path = input("Enter the path to your .ipynb file directory: ")
if not os.path.exists(notebook_path): sys.exit(f"Path {notebook_path} not found")
os.chdir(notebook_path)

Mounted at /content/drive
Enter the path to your .ipynb file directory: /content/drive/MyDrive/LJMU_Data_XAI/TestGItCode/whitebox_mlops/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
"""
Chunk Trainer - Complete Machine Learning Pipeline

Author: Prem Kumar Subudhi
Date: 11-Oct-2025
Version: 1.0

Description:
This script implements an advanced chunk-based training pipeline for multiclass classification.
It trains multiple models across data chunks, evaluates performance, and selects the best
performing model for deployment.

Processing Steps:
1. CONFIGURATION LOADING: Loads training parameters and model configurations from YAML file
2. DATA INGESTION: Reads balanced dataset and auto-detects feature/label columns
3. FEATURE ENGINEERING: Applies TF-IDF vectorization to transform text data into numerical features
4. MODEL INITIALIZATION: Prepares 5 different classification algorithms with configured hyperparameters
5. DATA CHUNKING: Partitions dataset into multiple chunks for distributed training
6. PARALLEL TRAINING: Trains all models across all chunks with performance tracking
7. MODEL PERSISTENCE: Saves all trained models with associated preprocessing components
8. PERFORMANCE EVALUATION: Ranks models based on primary metrics and selects optimal performer
9. MODEL SELECTION: Identifies and saves the best overall model for inference

Dependencies:
- pandas, numpy, scikit-learn, pyyaml
"""

# ChunkTrainer.py
import pandas as pd
import numpy as np
import pickle
import os
import yaml
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # Added import
from sklearn.tree import DecisionTreeClassifier # Added import
from sklearn.svm import SVC # Added import
import warnings
warnings.filterwarnings('ignore')

class ChunkTrainer:
    def __init__(self, config_path="config.yaml"):
        self.config = self.load_config(config_path)
        self.balanced_df = None
        self.chunks = []
        self.models = {
            'logistic_regression': {},
            'decision_tree': {},
            'neural_network': {},
            'svm': {},
            'lasso_regression': {}
        }
        self.metrics = {}
        self.label_encoder = LabelEncoder()
        self.vectorizer = None
        self.feature_columns = None
        self.best_chunk_models = {}
        self.overall_best_model = None

    def load_config(self, config_path):
        """Load configuration from YAML file"""
        try:
            with open(config_path, 'r') as file:
                config = yaml.safe_load(file)
            print(f"Configuration loaded successfully from {config_path}")
            return config
        except Exception as e:
            print(f"Error loading config from {config_path}: {e}")
            return self.get_default_config()

    def get_default_config(self):
        """Provide default configuration"""
        return {
            'models': {
                'logistic_regression': {
                    'multi_class': 'multinomial', 'solver': 'lbfgs', 'max_iter': 1000, 'random_state': 42, 'class_weight': 'balanced'
                },
                'decision_tree': {
                    'max_depth': 10, 'random_state': 42, 'class_weight': 'balanced'
                },
                'neural_network': {
                    'hidden_layer_sizes': [100, 50], 'max_iter': 1000, 'random_state': 42, 'early_stopping': True
                },
                'svm': {
                    'kernel': 'linear', 'probability': True, 'random_state': 42, 'class_weight': 'balanced'
                },
                'lasso_regression': {
                    'multi_class': 'multinomial', 'solver': 'saga', 'max_iter': 1000, 'random_state': 42, 'class_weight': 'balanced', 'penalty': 'l1'
                }
            },
            'training': {
                'num_chunks': 10, 'random_state': 42, 'primary_metric': 'f1_score'
            },
            'paths': {
                'balanced_dataset': "balanced_dataset", 'chunks_dir': "chunks",
                'models_dir': "models", 'best_models_dir': "best_models", 'reports_dir': "reports"
            }
        }


    def load_balanced_dataset(self):
        """Load the balanced dataset"""
        balanced_path = self.config['paths']['balanced_dataset']
        try:
            # Try loading from CSV
            csv_path = os.path.join(balanced_path, "balanced_dataset.csv")
            if os.path.exists(csv_path):
                self.balanced_df = pd.read_csv(csv_path)
                print(f"Loaded balanced dataset from: {csv_path}")
                print(f"Dataset shape: {self.balanced_df.shape}")
                return True
            else:
                print(f"Balanced dataset not found at: {csv_path}")
                return False

        except Exception as e:
            print(f"Error loading balanced dataset: {e}")
            return False

    def _auto_detect_label_column(self):
        """Auto-detect the label column"""
        label_candidates = ['label', 'category', 'class', 'target', 'industry', 'industry_name']
        for candidate in label_candidates:
            if candidate in self.balanced_df.columns:
                return candidate
        return self.balanced_df.columns[0] if len(self.balanced_df.columns) > 0 else None

    def _auto_detect_text_column(self):
        """Auto-detect the text column"""
        text_candidates = ['text', 'content', 'summary', 'description', 'full_summary']
        for candidate in text_candidates:
            if candidate in self.balanced_df.columns:
                return candidate
        for col in self.balanced_df.columns:
            if self.balanced_df[col].dtype == 'object' and self.balanced_df[col].str.len().mean() > 10:
                return col
        return self.balanced_df.columns[0] if len(self.balanced_df.columns) > 0 else None

    def prepare_features(self):
        """Prepare features using TF-IDF"""
        text_column = self._auto_detect_text_column()
        label_column = self._auto_detect_label_column()

        if not text_column or not label_column:
            print("Could not auto-detect text or label column")
            print(f"Available columns: {self.balanced_df.columns.tolist()}")
            return False

        self.text_column = text_column
        self.label_column = label_column

        print(f"Using text column: '{text_column}'")
        print(f"Using label column: '{label_column}'")

        # Get TF-IDF parameters from config
        tfidf_config = self.config.get('feature_engineering', {}).get('tfidf', {
            'max_features': 1000,
            'stop_words': 'english',
            'ngram_range': [1, 2],
            'min_df': 1,
            'max_df': 1.0
        })

        texts = self.balanced_df[text_column].fillna('').astype(str)

        self.vectorizer = TfidfVectorizer(
            max_features=tfidf_config['max_features'],
            stop_words=tfidf_config['stop_words'],
            ngram_range=tuple(tfidf_config['ngram_range']),
            min_df=tfidf_config['min_df'],
            max_df=tfidf_config['max_df']
        )


        X = self.vectorizer.fit_transform(texts)
        self.feature_columns = self.vectorizer.get_feature_names_out()

        # Prepare labels
        y = self.balanced_df[label_column]
        self.label_encoder = LabelEncoder()
        y_encoded = self.label_encoder.fit_transform(y)


        self.X = X
        self.y = y_encoded

        print(f"Created features with shape: {X.shape}")
        print(f"Number of classes: {len(self.label_encoder.classes_)}")
        print(f"Classes: {list(self.label_encoder.classes_)}")


        return True


    def initialize_models(self):
        """Initialize all 5 models with parameters from config"""
        models_config = self.config['models']

        # Logistic Regression
        lr_config = models_config['logistic_regression']
        self.models['logistic_regression'] = {
            'model_class': LogisticRegression,
            'config': lr_config,
            'chunk_models': {},
            'best_chunk': None,
            'best_score': -1
        }

        # Decision Tree
        dt_config = models_config['decision_tree']
        self.models['decision_tree'] = {
            'model_class': DecisionTreeClassifier,
            'config': dt_config,
            'chunk_models': {},
            'best_chunk': None,
            'best_score': -1
        }

        # Neural Network
        nn_config = models_config['neural_network']
        if 'hidden_layer_sizes' in nn_config and isinstance(nn_config['hidden_layer_sizes'], list):
            nn_config['hidden_layer_sizes'] = tuple(nn_config['hidden_layer_sizes'])
        self.models['neural_network'] = {
            'model_class': MLPClassifier,
            'config': nn_config,
            'chunk_models': {},
            'best_chunk': None,
            'best_score': -1
        }

        # SVM
        svm_config = models_config['svm']
        self.models['svm'] = {
            'model_class': SVC,
            'config': svm_config,
            'chunk_models': {},
            'best_chunk': None,
            'best_score': -1
        }

        # Lasso Regression (Logistic Regression with L1 penalty)
        lasso_config = models_config['lasso_regression']
        self.models['lasso_regression'] = {
            'model_class': LogisticRegression,
            'config': lasso_config,
            'chunk_models': {},
            'best_chunk': None,
            'best_score': -1
        }

        print("Initialized 5 models for multiclass classification:")
        for model_name in self.models.keys():
            print(f"  - {model_name}")

        return self.models

    def create_chunks(self):
        """Create chunks from the balanced dataset"""
        num_chunks = self.config['training']['num_chunks']

        if not hasattr(self, 'X') or self.X is None:
            print("Features not prepared.")
            return []

        total_samples = self.X.shape[0]
        chunk_size = total_samples // num_chunks
        remaining = total_samples % num_chunks

        print(f"\nCreating {num_chunks} chunks from {total_samples} samples...")

        self.chunks = []
        start_idx = 0

        for i in range(num_chunks):
            current_chunk_size = chunk_size + (1 if i < remaining else 0)
            end_idx = start_idx + current_chunk_size

            chunk_data = {
                'chunk_id': i + 1,
                'start_idx': start_idx,
                'end_idx': end_idx,
                'size': current_chunk_size,
                'X': self.X[start_idx:end_idx],
                'y': self.y[start_idx:end_idx],
                'class_distribution': dict(pd.Series(self.y[start_idx:end_idx]).value_counts())
            }

            self.chunks.append(chunk_data)
            print(f"Chunk {i+1}: {current_chunk_size} samples, {len(chunk_data['class_distribution'])} classes")

            start_idx = end_idx


        return self.chunks

    def train_models_on_chunk(self, chunk):
        """Train all 5 models on a specific chunk"""
        chunk_id = chunk['chunk_id']
        X_chunk = chunk['X']
        y_chunk = chunk['y']

        print(f"\n{'='*60}")
        print(f"TRAINING ALL 5 MODELS ON CHUNK {chunk_id}")
        print(f"{'='*60}")

        chunk_metrics = {}

        for model_name, model_info in self.models.items():
            print(f"\n--- Training {model_name.upper()} on chunk {chunk_id} ---")

            try:
                # Create fresh model instance
                model_instance = model_info['model_class'](**model_info['config'])

                # Train the model
                model_instance.fit(X_chunk, y_chunk)

                # Make predictions
                y_pred = model_instance.predict(X_chunk)

                # Calculate metrics
                accuracy = accuracy_score(y_chunk, y_pred)
                f1 = f1_score(y_chunk, y_pred, average='weighted')
                precision = precision_score(y_chunk, y_pred, average='weighted', zero_division=0)
                recall = recall_score(y_chunk, y_pred, average='weighted', zero_division=0)

                metrics = {
                    'accuracy': float(accuracy),
                    'f1_score': float(f1),
                    'precision': float(precision),
                    'recall': float(recall),
                    'train_samples': X_chunk.shape[0]
                }

                # Store the trained model with vectorizer and encoder
                model_info['chunk_models'][chunk_id] = {
                    'model': model_instance,
                    'metrics': metrics,
                    'trained_at': datetime.now().isoformat(),
                    'vectorizer': self.vectorizer,  # Save vectorizer
                    'label_encoder': self.label_encoder,  # Save label encoder
                    'X': X_chunk,  # Save training data for drift analysis
                    'y': y_chunk   # Save labels for drift analysis
                }

                chunk_metrics[model_name] = metrics

                print(f"  SUCCESS - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

                # Update best chunk for this model
                primary_metric = self.config['training']['primary_metric']
                current_score = metrics[primary_metric]
                if current_score > model_info['best_score']:
                    model_info['best_score'] = current_score
                    model_info['best_chunk'] = chunk_id
                    print(f"  NEW BEST - Best chunk for {model_name}: {chunk_id} ({primary_metric}: {current_score:.4f})")


            except Exception as e:
                print(f"  ERROR training {model_name}: {e}")
                chunk_metrics[model_name] = {'error': str(e)}


        self.metrics[chunk_id] = chunk_metrics
        return chunk_metrics


    def save_all_chunk_models(self):
        """Save all models for all chunks with vectorizer and encoder"""
        models_dir = self.config['paths']['models_dir']
        os.makedirs(models_dir, exist_ok=True)

        total_saved = 0
        for model_name, model_info in self.models.items():
            model_dir = os.path.join(models_dir, model_name)
            os.makedirs(model_dir, exist_ok=True)

            for chunk_id, chunk_model in model_info['chunk_models'].items():
                filename = os.path.join(model_dir, f"chunk_{chunk_id}.pkl")
                try:
                    with open(filename, 'wb') as f:
                        pickle.dump(chunk_model, f)
                    total_saved += 1
                except Exception as e:
                    print(f"Error saving {model_name} chunk {chunk_id}: {e}")

        print(f"\nSaved {total_saved} chunk models across 5 model types")
        return total_saved

    def find_and_save_best_chunk_models(self):
        """Find and save the best chunk model for each model type"""
        best_models_dir = self.config['paths']['best_models_dir']
        os.makedirs(best_models_dir, exist_ok=True)

        print(f"\n{'='*50}")
        print("FINDING BEST CHUNK MODELS FOR EACH MODEL TYPE")
        print(f"{'='*50}")

        self.best_chunk_models = {}

        for model_name, model_info in self.models.items():
            best_chunk = model_info['best_chunk']
            if best_chunk and best_chunk in model_info['chunk_models']:
                best_model_data = model_info['chunk_models'][best_chunk]
                self.best_chunk_models[model_name] = best_model_data

                # Save best model
                filename = os.path.join(best_models_dir, f"best_{model_name}.pkl")
                try:
                    with open(filename, 'wb') as f:
                        pickle.dump(best_model_data, f)

                    metrics = best_model_data['metrics']
                    primary_metric = self.config['training']['primary_metric']
                    primary_metric_value = metrics[primary_metric]
                    print(f"SUCCESS - {model_name:20} | Best chunk: {best_chunk:2} | {primary_metric}: {primary_metric_value:.4f} | Accuracy: {metrics['accuracy']:.4f}")

                except Exception as e:
                    print(f"Error saving best {model_name}: {e}")
            else:
                print(f"FAILED - {model_name:20} | No best chunk found")

        return self.best_chunk_models


    def rank_models(self):
        """Rank all models based on performance metrics"""
        if not self.best_chunk_models:
            print("No best models found for ranking")
            return []

        primary_metric = self.config['training']['primary_metric']

        # Calculate average metrics across all chunks for each model
        model_scores = []

        for model_name in self.models.keys():
            model_metrics = []
            for chunk_id, chunk_metrics in self.metrics.items():
                if model_name in chunk_metrics and primary_metric in chunk_metrics[model_name]:
                    model_metrics.append(chunk_metrics[model_name][primary_metric])

            if model_metrics:
                avg_score = np.mean(model_metrics)
                std_score = np.std(model_metrics)

                # Get best chunk performance
                best_score = self.models[model_name]['best_score']
                best_chunk = self.models[model_name]['best_chunk']

                model_scores.append({
                    'model': model_name,
                    f'avg_{primary_metric}': avg_score,
                    f'std_{primary_metric}': std_score,
                    f'best_{primary_metric}': best_score,
                    'best_chunk': best_chunk,
                    'chunks_trained': len(model_metrics)
                })


        # Sort by primary metric (descending)
        model_scores.sort(key=lambda x: x[f'avg_{primary_metric}'], reverse=True)


        return model_scores


    def print_model_ranking(self):
        """Print ranked list of models"""
        model_ranking = self.rank_models()
        primary_metric = self.config['training']['primary_metric']

        print(f"\n{'='*60}")
        print(f"MODEL RANKING (by {primary_metric})")
        print(f"{'='*60}")

        for i, model_data in enumerate(model_ranking, 1):
            avg_metric = model_data[f'avg_{primary_metric}']
            std_metric = model_data[f'std_{primary_metric}']
            best_metric = model_data[f'best_{primary_metric}']
            best_chunk = model_data['best_chunk']
            chunks_trained = model_data['chunks_trained']

            print(f"{i:2}. {model_data['model']:20} | "
                  f"Avg {primary_metric}: {avg_metric:.4f} (±{std_metric:.4f}) | "
                  f"Best: {best_metric:.4f} (chunk {best_chunk}) | "
                  f"Chunks: {chunks_trained}")


        return model_ranking


    def select_and_save_overall_best_model(self):
        """Select and save the overall best model for future inference"""
        model_ranking = self.rank_models()

        if not model_ranking:
            print("No models available for selection")
            return None

        best_model_data = model_ranking[0]
        best_model_name = best_model_data['model']
        best_chunk = self.models[best_model_name]['best_chunk']

        if best_chunk in self.models[best_model_name]['chunk_models']:
            self.overall_best_model = self.models[best_model_name]['chunk_models'][best_chunk]

            # Save overall best model
            best_models_dir = self.config['paths']['best_models_dir']
            filename = os.path.join(best_models_dir, "overall_best_model.pkl")

            try:
                # Extract the primary metric value safely
                primary_metric_name = self.config['training']['primary_metric']
                best_metric_value = best_model_data[f'best_{primary_metric_name}']


                with open(filename, 'wb') as f:
                    pickle.dump({
                        'model': self.overall_best_model['model'],
                        'model_name': best_model_name,
                        'best_chunk': best_chunk,
                        'metrics': self.overall_best_model['metrics'],
                        'primary_metric': primary_metric_name,
                        'selected_at': datetime.now().isoformat(),
                        'feature_columns': self.feature_columns.tolist() if hasattr(self, 'feature_columns') else [],
                        'label_encoder': self.label_encoder,
                        'vectorizer': self.vectorizer
                    }, f)

                print(f"\nOVERALL BEST MODEL SELECTED: {best_model_name}")
                print(f"   Best chunk: {best_chunk}")
                print(f"   {primary_metric_name}: {best_metric_value:.4f}")
                print(f"   Saved to: {filename}")


                return self.overall_best_model

            except Exception as e:
                print(f"Error saving overall best model: {e}")
                return None

        return None


    def run_complete_pipeline(self):
        """Run the complete training pipeline"""
        print("STARTING COMPLETE TRAINING PIPELINE")
        print("=" * 50)

        # Step 1: Load balanced dataset
        if not self.load_balanced_dataset():
            return False

        # Step 2: Prepare features
        if not self.prepare_features():
            return False

        # Step 3: Create chunks
        self.create_chunks()

        # Step 4: Initialize models
        self.initialize_models()

        # Step 5: Train on all chunks
        for chunk in self.chunks:
            self.train_models_on_chunk(chunk)

        # Step 6: Save all chunk models
        self.save_all_chunk_models()

        # Step 7: Find and save best chunk models
        self.find_and_save_best_chunk_models()

        # Step 8: Rank models and print results
        self.print_model_ranking()

        # Step 9: Select and save overall best model
        self.select_and_save_overall_best_model()

        print(f"\nPIPELINE COMPLETED SUCCESSFULLY!")
        print(f"   Models saved in: {self.config['paths']['models_dir']}/")
        print(f"   Best models saved in: {self.config['paths']['best_models_dir']}/")
        print(f"   Total chunks processed: {len(self.chunks)}")
        print(f"   Total models trained: {len(self.models)}")


        return True


def main():
    trainer = ChunkTrainer("config.yaml")
    success = trainer.run_complete_pipeline()

    if success:
        print("\nFINAL RESULTS SUMMARY:")
        print("=" * 40)


if __name__ == "__main__":
    main()

Configuration loaded successfully from config.yaml
STARTING COMPLETE TRAINING PIPELINE
Loaded balanced dataset from: balanced_dataset/balanced_dataset.csv
Dataset shape: (272, 12)
Using text column: 'full_summary'
Using label column: 'industry'
Created features with shape: (272, 1000)
Number of classes: 16
Classes: ['Automotive', 'Consulting', 'E-commerce', 'Education', 'Energy', 'Finance', 'Government', 'HR', 'Healthcare', 'Insurance', 'Legal', 'Media & Entertainment', 'Other', 'Research & Academia', 'Tech', 'Telecommunications']

Creating 10 chunks from 272 samples...
Chunk 1: 28 samples, 15 classes
Chunk 2: 28 samples, 12 classes
Chunk 3: 27 samples, 13 classes
Chunk 4: 27 samples, 13 classes
Chunk 5: 27 samples, 14 classes
Chunk 6: 27 samples, 14 classes
Chunk 7: 27 samples, 13 classes
Chunk 8: 27 samples, 14 classes
Chunk 9: 27 samples, 15 classes
Chunk 10: 27 samples, 13 classes
Initialized 5 models for multiclass classification:
  - logistic_regression
  - decision_tree
  - neur